In [15]:
import subprocess 
import json
import time, os
import struct, socket
import numpy as np
import xml.etree.ElementTree as ET
import sys
import linecache, bisect

In [10]:
#variables
iploc = '/home/hadoop/ipython/iploc/iploc.csv'

#----------------- GTI Integration------------------------#
rest_client_path = '/home/hadoop/refclient/restclient'
gti_server = 'preprod.rest.gti.mcafee.com'
gti_user= 'int-dux-dev'
gti_password = '0d52ar50D{6cszTM!q<8V<Iw4GC52/'

gti_command = (rest_client_path + ' -s '+ gti_server 
               + ' -q \'{"ci":{"cliid":"87d8d1082c2f2f821f438b2359b7a5b4", "prn":"Duxbury", "sdkv":"1.0", "pv":"1.0.0", "pev":1, "rid":1, "affid":"0"},"q":[{"op":"ip","ip":"###IP###"}]}\''
               + ' -i '+ gti_user + ' -p \'' + gti_password + '\' -t')

#------------------ Norse---------------------------------#
norse_api_key = 'b07ce7f6cf9ebc8b714422b25614b60ea8114efd97b66c7590e1da3609a7ed3c'
norse_url = 'http://us.api.ipviking.com/api/'
norse_command = 'curl -d apikey={0} -d method=ipview -d ip={1} {2}'



print 'loading context....'
iplist = np.loadtxt(iploc,dtype=np.uint32,delimiter=',',usecols={0},
                    converters={0: lambda s: np.uint32(s.replace('"',''))})


loading context....


# Functions Definition

In [25]:
def ip_to_int(ip):
    o = map(int, ip.split('.'))
    res = (16777216 * o[0]) + (65536 * o[1]) + (256 * o[2]) + o[3]
    return res
def check_if_ip_is_internal(ip, ranges):
    for r in ranges:
        if(ip_to_int(ip) >= r[0] and ip_to_int(ip) <= r[1]):
            return 1
    return 0

def get_geo_ip(ip):
    result = linecache.getline(iploc, bisect.bisect(iplist,ip_to_int(ip))).replace('\n','')
    return result

def get_gti_rep(ip):
    reputation = ''
    try:
        response_json = subprocess.check_output(gti_command.replace('###IP###', ip), shell=True)
        result_dict = json.loads(response_json[0:len(response_json) - 1])
        reputation = result_dict['a'][0]['rep']
    except:
        reputation = 'error calling GTI'
    return reputation

#Currently it only returns the risk name... we can expand it to return a hash of interesting values
def get_norse_rep(ip):
    reputation = ''
    try:
        response = subprocess.check_output(norse_command.format(norse_api_key, ip, norse_url), shell=True)
        # the executed curl command returns an XML as response 
        root_element = ET.fromstring(response)
        reputation = root_element.find("./response/risk_name").text
    except:
        reputation = 'error calling norse'
    
    return reputation


In [26]:
#Adds the network internal IP ranges to a matrix
ranges_file_path = 'IntelIP_Ranges.csv'
ranges = []

f = open(ranges_file_path, 'r')

for line in f:
    line_parts = line.split(',');
    ranges.append([ip_to_int(line_parts[0]), ip_to_int(line_parts[1])])

print ranges
f.close()

[[2885681152, 2902458367]]


In [27]:
lda_file = '/home/hadoop/ipython/vast/user/vast/lda_scores.csv'
dest = '/home/hadoop/ipython/vast/user/vast/lda_scores_bu.csv'
temp = '/home/hadoop/ipython/vast/user/vast/lda_scores_tmp.csv'
#!cp $lda_file $dest
!cp $lda_file $temp



In [28]:
#import csv
ip_dict = {}
new_content = ''
with open(dest, 'r+') as lda_f:
    for line in lda_f:    
        if(new_content==""):
              
            new_content += line[:-1] 
            new_content += ',srcIpInternal,destIpInternal,srcGeo,dstGeo,srcDomain,dstDomain'
            #new_content += ',gtiSrcRep,gtiDstRep'
            #new_content += ',norseSrcRep,norseDstRep'
            new_content += '\n'
            
        else:
            line_parts = line.split(',')
            src_ip = line_parts[3]
            dst_ip = line_parts[4]
            is_src_ip_internal = check_if_ip_is_internal(src_ip, ranges) # index 2 is the srcIP
            is_dest_ip_internal = check_if_ip_is_internal(dst_ip, ranges) # index 3 is the destip           
            
            #Add current line content... chop '\n' character
            new_content += line[:-1] 
            
            #---- append is Internal flags
            new_content += ',' + str(is_src_ip_internal) + ',' + str(is_dest_ip_internal) 
            
            src_geo = ";".join(get_geo_ip(src_ip).replace('"','').split(',')[4:6])+ " " + ";".join(get_geo_ip(src_ip).replace('"','').split(',')[8:9])
            dst_geo = ";".join(get_geo_ip(dst_ip).replace('"','').split(',')[4:6]) + " " + ";".join(get_geo_ip(dst_ip).replace('"','').split(',')[8:9])            
            src_domain = get_geo_ip(src_ip).replace('"','').split(',')[9:10][0]
            dst_domain = get_geo_ip(dst_ip).replace('"','').split(',')[9:10][0]            
            #check if src IP is not in dictionary and add it if true
            
            if src_ip not in ip_dict:                                
                if is_src_ip_internal:
                    ip_dict[src_ip] = { 'geo': src_geo
                                       ,'domain': src_domain 
                                       #,'gti_rep': ''
                                       #,'norse_rep': ''
                                      }
                else:                   
                    ip_dict[src_ip] = { 'geo': src_geo
                                       ,'domain': src_domain
                                       #,'gti_rep': get_gti_rep(src_ip)
                                       #,'norse_rep': get_norse_rep(src_ip) 
                                      }                   
            
            if dst_ip not in ip_dict:
                if is_dest_ip_internal:                    
                    ip_dict[dst_ip] = { 'geo': dst_geo
                                       ,'domain': dst_domain
                                       #,'gti_rep': ''
                                       #,'norse_rep': ''
                                      }
                else:                   
                    ip_dict[dst_ip] = { 'geo': dst_geo
                                       ,'domain': dst_domain
                                       #,'gti_rep': get_gti_rep(dst_ip)
                                       #,'norse_rep':get_norse_rep(dst_ip) 
                                      }
                   
            
            #print src_ip
            #print get_geo_ip(src_ip)
            
            #------Add Geo Location-------------#
            new_content = new_content + "," + ip_dict[src_ip]['geo'] + ',' +ip_dict[dst_ip]['geo']
            #------Add Domain-------------------#
            new_content = new_content + "," + ip_dict[src_ip]['domain'] + ',' +ip_dict[dst_ip]['domain']
            
            #--------Add GTI REP                                   
            #print str(ip_dict[src_ip]['gti_rep'])
            #print str(ip_dict[dst_ip]['gti_rep'])
            #new_content += "," +  str(ip_dict[src_ip]['gti_rep']) + ',' +  str(ip_dict[dst_ip]['gti_rep'])
            
            #-------Add Norse Rep
            #new_content += "," +  ip_dict[src_ip]['norse_rep'] + ',' +  ip_dict[dst_ip]['norse_rep']
            
            
            new_content += "\n"
            
print new_content
        
    

sev,score,Date,srcIP,dstIP,SPort,DPort,Sfirst,Dfirst,Stotal,Dtotal,Spkts,Dpkts,srcIpInternal,destIpInternal,srcGeo,dstGeo,srcDomain,dstDomain
0,1.50E-07,4/14/2013 15:10,10.70.68.127,172.30.0.4,6395,80,0,0,278,116,5,2,0,1,-;- Private IP Address LAN,-;- Private IP Address LAN,-,-
0,1.50E-07,4/14/2013 15:09,10.70.68.127,172.30.0.4,55759,80,19,1006,297,1114,5,2,0,1,-;- Private IP Address LAN,-;- Private IP Address LAN,-,-
0,1.51E-07,4/10/2013 17:16,10.13.77.49,172.10.0.40,61783,80,0,0,148,108,2,2,0,1,-;- Private IP Address LAN,Texas;Richardson Laurie Hatfield DBA,-,sbcglobal.net
0,1.51E-07,4/14/2013 15:16,10.13.77.49,172.10.0.2,0,0,0,0,354,354,2,2,0,1,-;- Private IP Address LAN,California;San Diego Gregory Hout Attorney at Law,-,sbcglobal.net
0,1.51E-07,4/14/2013 17:16,10.13.77.49,172.10.0.40,47131,80,0,0,206,162,3,3,0,1,-;- Private IP Address LAN,Texas;Richardson Laurie Hatfield DBA,-,sbcglobal.net
0,1.51E-07,4/14/2013 14:56,10.13.77.49,172.10.0.3,35579,25,0,0,112,58,2,1,0,1,-;- Private I

In [29]:
f = open(dest, 'w')
f.write(new_content)
f.close()

f2 = open(lda_file, 'w')
f2.write(new_content)
f2.close()
